In [2]:
from google.colab import drive
# Mount the Google Drive at mount
mount='/content/gdrive'
print("Colab: mounting Google drive on ", mount)

drive.mount(mount)

# Switch to the directory on the Google Drive that you want to use
import os
drive_root = mount + "/My Drive/Colab Notebooks/certainlyUncertain/models"

# Change to the directory
print("\nColab: Changing directory to ", drive_root)
%cd $drive_root


Colab: mounting Google drive on  /content/gdrive
Mounted at /content/gdrive

Colab: Changing directory to  /content/gdrive/My Drive/Colab Notebooks/certainlyUncertain/models
/content/gdrive/My Drive/Colab Notebooks/certainlyUncertain/models


In [3]:
!pip install pmdarima

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 8.1 MB/s 
     |████████████████████████████████| 9.9 MB 46.3 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.12.2
    Uninstalling statsmodels-0.12.2:
      Successfully uninstalled statsmodels-0.12.2


In [4]:
import pandas as pd
import numpy as np
import os
import statsmodels.api as sm
from pmdarima.arima import ADFTest
import pmdarima
from matplotlib import pyplot
import logging

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_percentage_error

import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
warnings.filterwarnings("ignore")

# DATA

In [55]:
df_train_ec = pd.read_hdf('../preprocessing/data/EC15_split.h5', key="train")
df_valid_ec = pd.read_hdf('../preprocessing/data/EC15_split.h5', key="validation")
df_test_ec = pd.read_hdf('../preprocessing/data/EC15_split.h5', key="test")

gbr_temp_cols = [col for col in df_train_ec.columns if 'GBR_TEMP' in col]
df_train_ec.drop(columns=gbr_temp_cols)
#df_train = pd.concat([df_train_ec, df_valid_ec])
df_train_ec.sort_index(inplace=True)

#get columns that end with 24 and have std in them
interesting_cols_ec = [col for col in df_train_ec.columns if col.endswith('24') and 'std' in col]
exog_cols_ec = [col for col in df_train_ec.columns if col.endswith('48') and 'std' in col]
exog_cols_ec2 = [col for col in df_train_ec.columns if col.endswith('72') and 'std' in col]

In [64]:
df_train_eg = pd.read_hdf('../preprocessing/data/GEFS_split.h5', key="train")
df_valid_eg = pd.read_hdf('../preprocessing/data/GEFS_split.h5', key="validation")
df_test_eg = pd.read_hdf('../preprocessing/data/GEFS_split.h5', key="test")

#df_train = pd.concat([df_train_eg, df_valid_eg])
df_train_eg.sort_index(inplace=True)

#get columns that end with 24 and have std in them
interesting_cols_eg = [col for col in df_train_eg.columns if col.endswith('24') and 'std' in col]
exog_cols_eg = [col for col in df_train_eg.columns if col.endswith('48') and 'std' in col]
exog_cols_eg2 = [col for col in df_train_eg.columns if col.endswith('72') and 'std' in col]

In [65]:
df_train_eg.dropna(inplace=True, axis='columns')
df_valid_eg.dropna(inplace=True,axis='columns')
df_test_eg.dropna(inplace=True,axis='columns')
df_train_eg.isna().sum()[df_train_eg.isna().sum() == 1]

Series([], dtype: int64)

In [45]:
df_train_eg.isna().sum()[df_train_eg.isna().sum() == 1]

model_0_value-DEU_TEMP_GEFS-48    1
model_0_value-DEU_TEMP_GEFS-72    1
ens_std-DEU_TEMP_GEFS-48          1
ens_std-DEU_TEMP_GEFS-72          1
model_0_value-NLD_TEMP_GEFS-48    1
model_0_value-NLD_TEMP_GEFS-72    1
ens_std-NLD_TEMP_GEFS-48          1
ens_std-NLD_TEMP_GEFS-72          1
model_0_value-FRA_TEMP_GEFS-48    1
model_0_value-FRA_TEMP_GEFS-72    1
ens_std-FRA_TEMP_GEFS-48          1
ens_std-FRA_TEMP_GEFS-72          1
dtype: int64

In [9]:
print("Size of the training set EC: ", df_train_ec.shape)
print("Size of the validation set EC: ", df_valid_ec.shape)
print("Size of the test set EC: ", df_test_ec.shape)

Size of the training set EC:  (3989, 56)
Size of the validation set EC:  (2393, 56)
Size of the test set EC:  (1597, 56)


In [10]:
print("Size of the training set EG: ", df_train_eg.shape)
print("Size of the validation set EG: ", df_valid_eg.shape)
print("Size of the test set EG: ", df_test_eg.shape)

Size of the training set EG:  (8046, 64)
Size of the validation set EG:  (4827, 64)
Size of the test set EG:  (3220, 64)


In [11]:
cols_ec = df_train_ec.columns
cols_ec

Index(['model_0_value-FRA_WIND_EC15-0', 'model_0_value-FRA_WIND_EC15-24',
       'model_0_value-FRA_WIND_EC15-48', 'model_0_value-FRA_WIND_EC15-72',
       'ens_std-FRA_WIND_EC15-0', 'ens_std-FRA_WIND_EC15-24',
       'ens_std-FRA_WIND_EC15-48', 'ens_std-FRA_WIND_EC15-72',
       'model_0_value-NLD_WIND_EC15-0', 'model_0_value-NLD_WIND_EC15-24',
       'model_0_value-NLD_WIND_EC15-48', 'model_0_value-NLD_WIND_EC15-72',
       'ens_std-NLD_WIND_EC15-0', 'ens_std-NLD_WIND_EC15-24',
       'ens_std-NLD_WIND_EC15-48', 'ens_std-NLD_WIND_EC15-72',
       'model_0_value-GBR_WIND_EC15-0', 'model_0_value-GBR_WIND_EC15-24',
       'model_0_value-GBR_WIND_EC15-48', 'model_0_value-GBR_WIND_EC15-72',
       'ens_std-GBR_WIND_EC15-0', 'ens_std-GBR_WIND_EC15-24',
       'ens_std-GBR_WIND_EC15-48', 'ens_std-GBR_WIND_EC15-72',
       'model_0_value-DEU_WIND_EC15-0', 'model_0_value-DEU_WIND_EC15-24',
       'model_0_value-DEU_WIND_EC15-48', 'model_0_value-DEU_WIND_EC15-72',
       'ens_std-DEU_WIND_EC15

In [12]:
cols_eg = df_train_eg.columns
cols_eg

Index(['model_0_value-DEU_WIND_GEFS-0', 'model_0_value-DEU_WIND_GEFS-24',
       'model_0_value-DEU_WIND_GEFS-48', 'model_0_value-DEU_WIND_GEFS-72',
       'ens_std-DEU_WIND_GEFS-0', 'ens_std-DEU_WIND_GEFS-24',
       'ens_std-DEU_WIND_GEFS-48', 'ens_std-DEU_WIND_GEFS-72',
       'model_0_value-GBR_WIND_GEFS-0', 'model_0_value-GBR_WIND_GEFS-24',
       'model_0_value-GBR_WIND_GEFS-48', 'model_0_value-GBR_WIND_GEFS-72',
       'ens_std-GBR_WIND_GEFS-0', 'ens_std-GBR_WIND_GEFS-24',
       'ens_std-GBR_WIND_GEFS-48', 'ens_std-GBR_WIND_GEFS-72',
       'model_0_value-NLD_WIND_GEFS-0', 'model_0_value-NLD_WIND_GEFS-24',
       'model_0_value-NLD_WIND_GEFS-48', 'model_0_value-NLD_WIND_GEFS-72',
       'ens_std-NLD_WIND_GEFS-0', 'ens_std-NLD_WIND_GEFS-24',
       'ens_std-NLD_WIND_GEFS-48', 'ens_std-NLD_WIND_GEFS-72',
       'model_0_value-FRA_WIND_GEFS-0', 'model_0_value-FRA_WIND_GEFS-24',
       'model_0_value-FRA_WIND_GEFS-48', 'model_0_value-FRA_WIND_GEFS-72',
       'ens_std-FRA_WIND_GEFS

### Test for stationarity

In [ ]:
for col in uncertainty_columns:
    try:
        if ADFTest(alpha=0.05).should_diff(dataTrain[col])[1] == True:
            print("Column {} is not stationary".format(col))
    except BaseException as exception:
        print("Column {} has error: {}".format(col, exception))

### Trying to predict out of sample uncertainty

In [ ]:
uncertainty_columns = ['ens_std-DEU_TEMP_EC15-138','mae-DEU_TEMP_EC15-30']
for col in uncertainty_columns:
    model = ARIMA(dataTrain[col], order=(1,0,0))
    model_fit = model.fit()
    model_pred = model_fit.predict(start=np.size(dataTrain,0), end=np.size(dataTrain,0)+np.size(dataVal,0), typ='levels')[1:]
    model_pred.index = dataVal.index

    # plot results for the training set
    pyplot.plot(dataTrain[col], label='actual')
    pyplot.plot(model_fit.fittedvalues, label='predicted')
    pyplot.legend()
    #save the plots
    pyplot.savefig(os.path.join(os.path.dirname(os.getcwd()), 'models', 'plots', 'univariate', 'training', col + '.png'))
    pyplot.show()

    # plot results for the validation set
    pyplot.plot(dataVal[col], label='actual')
    pyplot.plot(model_pred, label='predicted')
    pyplot.legend()
    #save the plots
    pyplot.savefig(os.path.join(os.path.dirname(os.getcwd()), 'models', 'plots', 'univariate', 'validation', col + '.png'))
    pyplot.show()

##### In the validation set, the model converges to the long term mean as expected

### Simply predicting one step ahead

In [ ]:
uncertainty_columns = ['ens_std-DEU_TEMP_EC15-138','mae-DEU_TEMP_EC15-30']
data = dataTrain.append(dataVal)
nTrain = np.size(dataTrain,0)

for col in uncertainty_columns:

    valPred = []
    for t in range(len(dataVal)):
        model = ARIMA(data[col][:nTrain+t], order=(1,1,0))
        model_fit = model.fit()
        yhat = model_fit.forecast()
        valPred.append(float(yhat))
    dataVal[col + '_pred'] = valPred

    # plot results for the validation set
    pyplot.plot(dataVal[col], label='actual')
    pyplot.plot(dataVal[col + '_pred'], label='predicted')
    pyplot.legend()
    #save the plots
    pyplot.savefig(os.path.join(os.path.dirname(os.getcwd()), 'models', 'plots', 'univariate', 'validation', col + '.png'))
    pyplot.show()
    

In [ ]:
# calclulate the mean squared error for the validation set
for col in uncertainty_columns:
    mse = np.mean((dataVal[col].values - dataVal[col + '_pred'].values)**2)
    print("The mean squared error for {} is: {}".format(col, mse))

### Predicting one step ahead for all columns

In [ ]:
def univariate_uncertainty_prediction(dataTrain, dataVal):
    std_columns = [col for col in dataTrain.columns if 'std' in col]
    mae_columns = [col for col in dataTrain.columns if 'mae' in col]
    uncertainty_columns = std_columns + mae_columns

    data[uncertainty_columns] = dataTrain[uncertainty_columns].append(dataVal[uncertainty_columns])
    nTrain = np.size(dataTrain,0)

    for col in uncertainty_columns:
        valPred = []
        for t in range(len(dataVal)):
            model = ARIMA(data[col][:nTrain+t], order=(1,0,0))
            model_fit = model.fit()
            yhat = model_fit.forecast()
            valPred.append(float(yhat))
        dataVal[col + '_pred'] = valPred
    
    return dataVal

In [ ]:
predVal = univariate_uncertainty_prediction(dataTrain, dataVal)

In [ ]:
std_columns = [col for col in dataTrain.columns if 'std' in col]
mae_columns = [col for col in dataTrain.columns if 'mae' in col]
uncertainty_columns = std_columns + mae_columns

In [ ]:
# calclulate the mean squared error for the validation set
mse_std = []
for col in std_columns:
    mse = np.mean((dataVal[col].values - dataVal[col + '_pred'].values)**2)
    mse_std.append(mse)

#plot the results
pyplot.plot(mse_std, label='mse_std')
pyplot.legend()
pyplot.show()

In [ ]:
mse_mae = []
for col in mae_columns:
    mse = np.mean((dataVal[col].values - dataVal[col + '_pred'].values)**2)
    mse_mae.append(mse)

#plot the results
pyplot.plot(mse_mae, label='mse_mae')
pyplot.legend()
pyplot.show()

### For specific ValueDate, see ensemble uncertainty vs predicted uncertainty for different ForecastHorizon

#### For Uncertainty measured as STD

In [ ]:
predVal_std = dataVal[[col for col in dataVal.columns if 'std' in col and '_pred' in col]]
dataVal_std = dataVal[[col for col in dataVal.columns if 'std' in col and '_pred' not in col]]
std_pred = [predVal_std.iloc[-1,0]]
std_true = [dataVal_std.iloc[-1,0]]
for iter in range(1,int(len(std_columns)/2)):
    row = np.size(dataVal,0) - iter - 1
    col = 2 * iter
    std_pred.append(predVal_std.iloc[row,col])
    std_true.append(dataVal_std.iloc[row,col])

In [ ]:
# plot std_pred vs std_true
x_axis = np.arange(12, 372, 12)
pyplot.plot(x_axis, std_pred[1:], label='std_pred')
pyplot.plot(x_axis, std_true[1:], label='std_true')
pyplot.legend()
#save the plots
pyplot.savefig(os.path.join(os.path.dirname(os.getcwd()), 'models', 'plots', 'univariate', 'validation', 'std_pred_vs_std_true.png'))
pyplot.show()

#### For Uncertainty measured as MAE

In [ ]:
predVal_mae = dataVal[[col for col in dataVal.columns if 'mae' in col and '_pred' in col]]
dataVal_mae = dataVal[[col for col in dataVal.columns if 'mae' in col and '_pred' not in col]]
mae_pred = [predVal_mae.iloc[-1,0]]
mae_true = [dataVal_mae.iloc[-1,0]]
for iter in range(1,int(len(std_columns)/2)):
    row = np.size(dataVal,0) - iter - 1
    col = 2 * iter
    mae_pred.append(predVal_mae.iloc[row,col])
    mae_true.append(dataVal_mae.iloc[row,col])

In [ ]:
# plot mae_pred vs mae_true
x_axis = np.arange(12, 372, 12)
pyplot.plot(x_axis, mae_pred[1:], label='predicted')
pyplot.plot(x_axis,mae_true[1:], label='actual')
pyplot.legend()
#save the plots
pyplot.savefig(os.path.join(os.path.dirname(os.getcwd()), 'models', 'plots', 'univariate', 'validation', 'mae_pred_vs_mae_true.png'))
pyplot.show()

# MODELS

In [ ]:
def train_predict_save_arima(train, valid, test, cols, order, window, save_path):
    
    data = train.append(valid)
    data = data.append(test)
    nTrain = np.size(train,0)
    nValid = np.size(valid,0)
    nTest = np.size(test,0)

    for col in cols:

        print("Validation Training and predicting for {}".format(col))
        y_valid_hat = []
        for t in range(nValid):
            model = sm.tsa.arima.ARIMA(data[col][nTrain+t-window:nTrain+t], order=(1,1,0))
            model_fit = model.fit()
            y_hat = model_fit.forecast(steps=2)[1]
            y_valid_hat.append(float(y_hat))
        valid[col + '_pred'] = y_valid_hat

        print("Test Training and predicting for {}".format(col))
        y_test_hat = []
        for t in range(nTest):
            model = sm.tsa.arima.ARIMA(data[col][nTrain+t-window:nTrain+nValid+t], order=(1,1,0))
            model_fit = model.fit()

            y_hat = model_fit.forecast(steps=2)[1]
            y_test_hat.append(float(y_hat))

            # save the model for the last few days fitted until this point with index of the last observation
            if t >= nTest-11:
                last_row = data[col].index[nTrain+nValid+t]
                model_fit.save(os.path.join(save_path, 'arima_model_' + col + '_' + str(last_row) + '.pkl'))
                
        test[col + '_pred'] = y_test_hat

    valid.to_csv(os.path.join(save_path, 'arima_valid.csv'))
    test.to_csv(os.path.join(save_path, 'arima_test.csv'))
    
    return valid, test

In [ ]:
def train_predict_save_arimax(train, valid, test, cols, ex_cols, order, window, save_path):
    
    data = train.append(valid)
    data = data.append(test)
    nTrain = np.size(train,0)
    nValid = np.size(valid,0)
    nTest = np.size(test,0)

    i = 0
    for col in cols:

        ex_col = ex_cols[i]
        print("Validation Training and predicting for {}".format(col))
        y_valid_hat = []
        for t in range(nValid):
            model = sm.tsa.arima.ARIMA(data[col][nTrain+t-window:nTrain+t], order=order, exog = data[ex_col][nTrain+t-window:nTrain+t] )
            model_fit = model.fit()
            y_hat = model_fit.forecast(steps=2, exog = data[ex_col][nTrain+t+1:nTrain+t+3])[1]
            y_valid_hat.append(float(y_hat))
        valid[col + '_pred'] = y_valid_hat

        print("Test Training and predicting for {}".format(col))
        y_test_hat = []
        for t in range(nTest-2):
            model = sm.tsa.arima.ARIMA(data[col][nTrain+nValid+t-window:nTrain+nValid+t], order=order, 
                                       exog = data[ex_col][nTrain+nValid+t-window:nTrain+nValid+t])
            model_fit = model.fit()

            y_hat = model_fit.forecast(steps=2, exog = data[ex_col][nTrain+nValid+t+1:nTrain+nValid+t+3])[1]
            y_test_hat.append(float(y_hat))

            # save the model for the last few days fitted until this point with index of the last observation
            if t >= nTest-11:
                last_row = data[col].index[nTrain+nValid+t]
                model_fit.save(os.path.join(save_path, 'arimax_model_' + col + '_' + str(last_row) + '.pkl'))

        test[col + '_pred'] = 0        
        test[col + '_pred'][:-2] = y_test_hat


        i += 1

    valid.to_csv(os.path.join(save_path, 'arimax_valid.csv'))
    test.to_csv(os.path.join(save_path, 'arimax_test.csv'))
    
    return valid, test

In [18]:
def train_predict_save_arimax2(train, valid, test, cols, ex_cols, order, window, save_path):
    
    data = train.append(valid)
    data = data.append(test)
    nTrain = np.size(train,0)
    nValid = np.size(valid,0)
    nTest = np.size(test,0)

    i = 0
    for col in cols:

        ex_col = ex_cols[i]
        print("Validation Training and predicting for {}".format(col))
        y_valid_hat = []
        for t in range(nValid):
            model = pmdarima.arima.ARIMA(order=order)
            model_fit = model.fit(data[col][nTrain+t-window:nTrain+t],exog = data[ex_col][nTrain+t+1-window:nTrain+t+1] )
            y_hat = model_fit.predict(nperiods=2, exog = data[ex_col][nTrain+t+1:nTrain+t+3])[1]
            y_valid_hat.append(float(y_hat))
        valid[col + '_pred'] = y_valid_hat

        """
        print("Test Training and predicting for {}".format(col))
        y_test_hat = []
        for t in range(nTest-2):
            model = sm.tsa.arima.ARIMA(data[col][nTrain+nValid+t-window:nTrain+nValid+t+1], order=order, 
                                       exog = data[ex_col][nTrain+nValid+t-window:nTrain+nValid+t+1])
            model_fit = model.fit()

            y_hat = model_fit.forecast(steps=2, exog = data[ex_col][nTrain+nValid+t+1:nTrain+nValid+t+3])[1]
            y_test_hat.append(float(y_hat))

            # save the model for the last few days fitted until this point with index of the last observation
            #if t >= nTest-11:
                #last_row = data[col].index[nTrain+nValid+t+1]
                #model_fit.save(os.path.join(save_path, 'arimax_model_' + col + '_' + str(last_row) + '.pkl'))

        test[col + '_pred'] = 0        
        test[col + '_pred'][:-2] = y_test_hat
        """

        i += 1

    valid.to_csv(os.path.join(save_path, 'arimax_valid.csv'))
    test.to_csv(os.path.join(save_path, 'arimax_test.csv'))
    
    return valid, test

In [19]:
def r2_calc(y_train, y, y_hat):
    sse = np.sum((y - y_hat)**2)
    sst = np.sum((y - y_train.mean())**2)
    return 1 - sse/sst

In [20]:
def evaluate_arima(valid, test, train, cols):
    from sklearn.metrics import r2_score
    #calculate r2 score for the validation set for each column
    r2_valid = []
    mape_valid = []
    for col in cols:
        tr = train[col]
        pred = valid[col + '_pred'].values[:-2]
        true = valid[col].shift(-2).values[:-2]
        #r2 = r2_score(true, pred)
        r2 = r2_calc(tr, true, pred)
        r2_valid.append(r2)
        mape = mean_absolute_percentage_error(true,pred)
        mape_valid.append(mape)
        #print r2 of the column
        print("Validation r2 score for {} is {}".format(col, r2))
        print("Validation mape score for {} is {}".format(col, mape))
    
    """
    #calculate r2 score for the test set
    r2_test = []
    mape_test = []
    for col in cols:
        pred = test[col + '_pred'].values[:-2]
        true = test[col].shift(-2).values[:-2]
        #r2 = r2_score(true, pred)
        r2 = r2_calc(train, true, pred)
        mape = mean_absolute_percentage_error(true,pred)
        mape_valid.append(mape)
        #print r2 of the column
        print("Test r2 score for {} is {}".format(col, r2))
        print("Test mape score for {} is {}".format(col, mape))
    """


    return r2_valid

# ARIMAX

In [50]:
save_path = os.path.join(os.path.dirname(os.getcwd()), 'models', 'model_save','arimax', 'gefs')
#look at last 5 years for training
window = 360 * 2

In [75]:
interesting_cols_eg[5]

'ens_std-GBR_TEMP_GEFS-24'

In [23]:
exog_cols = np.transpose(np.array([exog_cols_eg,exog_cols_eg2]))
np.shape(exog_cols)

(8, 2)

In [77]:
valid_eg, test_eg = train_predict_save_arimax2(df_train_eg, df_valid_eg, df_test_eg, [interesting_cols_eg[4]], [exog_cols[4]], (2,0,0), window, save_path)

Validation Training and predicting for ens_std-DEU_TEMP_GEFS-24


In [78]:
arimax_valid = pd.read_csv('model_save/arimax/gefs/arimax_valid.csv')
arimax_test = pd.read_csv('model_save/arimax/gefs/arimax_test.csv')

In [79]:
r2_valid_ec = evaluate_arima(arimax_valid,arimax_test, df_train_eg, [interesting_cols_eg[1]])

Validation r2 score for ens_std-GBR_WIND_GEFS-24 is 0.17794338935430576
Validation mape score for ens_std-GBR_WIND_GEFS-24 is 0.45290201455831247
